# Quick test of different ways of estimating Kappas across subejcts and parcels



In [1]:
# dependencies
%load_ext autoreload
%autoreload 2 
import os
import sys 
sys.path.append(os.path.abspath('..'))

import numpy as np
import torch as pt
import matplotlib.pyplot as plt
from copy import copy,deepcopy

import HierarchBayesParcel.emissions as em
import HierarchBayesParcel.arrangements as ar


## make some artifical data

In [2]:

n_subj = 4
P = 1000
N = 10
K = 5
U = pt.randint(low=0, high=K, size=(n_subj, P))
U_hat = ar.expand_mn(U,K)
emission_model = em.MixVMF(K, N, P, n_subj)
emission_model.kappa=pt.tensor(20)
Y = emission_model.sample(U)

In [3]:
m = deepcopy(emission_model)
m.random_params()

## Do an M-step by hand:

In [4]:
JU = pt.sum(U_hat,dim=2)   # (num_sub, K)
 # Calculate YU = \sum_i\sum_k<u_i^k>y_i # (num_sub, N, K)
YU = pt.matmul(Y, pt.transpose(U_hat, 1, 2)) 

# If the subjects are weighted differently
r_norm2 = pt.sum(YU ** 2, dim=1, keepdim=True) # (num_sub, 1, K)


In [5]:
pt.sqrt(r_norm2.squeeze())/JU

tensor([[0.7905, 0.7955, 0.7899, 0.7989, 0.8020],
        [0.7998, 0.7952, 0.8021, 0.7979, 0.7967],
        [0.7877, 0.7896, 0.8044, 0.7955, 0.7940],
        [0.8002, 0.8098, 0.7962, 0.7996, 0.7997]])

In [6]:
# Overall V estimation
V = pt.sum(YU,dim=0) / JU.sum(dim=0, keepdim=True)
print(pt.sqrt(pt.sum(V**2, dim=0, keepdim=True)))

tensor([[0.7940, 0.7970, 0.7973, 0.7970, 0.7973]])


In [7]:
# Subject-equal weighting V estimation
V=pt.nanmean(YU / JU.unsqueeze(1),dim=0)
print(pt.sqrt(pt.sum(V**2, dim=0, keepdim=True)))

tensor([[0.7938, 0.7970, 0.7974, 0.7971, 0.7974]])


In [8]:
# Overall kappa 
yu = pt.sum(YU,dim=0)
r=pt.sum(pt.sqrt(pt.sum(yu**2, dim=0)))/ pt.sum(JU)
kappa = (r*N - r**3) / (1 - r**2)
print(r, kappa)

tensor(0.7965) tensor(20.4098)


In [9]:
# Subject-specific kappa
r = pt.sum(pt.sqrt(pt.sum(YU**2, dim=1)),dim=1)/pt.sum(JU,dim=1)
kappa = (r*N - r**3) / (1 - r**2)
print(r, kappa)

tensor([0.7955, 0.7984, 0.7941, 0.8011]) tensor([20.2941, 20.6166, 20.1457, 20.9284])


In [10]:
# Region-specfic kappa
yu = pt.sum(YU,dim=0)
r=pt.sqrt(pt.sum(yu**2, dim=0))/ pt.sum(JU,dim=0)
kappa = (r*N - r**3) / (1 - r**2)
print(r, kappa)

tensor([0.7940, 0.7970, 0.7973, 0.7970, 0.7973]) tensor([20.1350, 20.4602, 20.4958, 20.4650, 20.4911])


In [ ]:
# Region and subject-specific kappa 
r=pt.sqrt(pt.sum(YU**2, dim=1))/ JU
kappa = (r*N - r**3) / (1 - r**2)
print(r, kappa)